# copy of generate_error.py from proPTV

In [1]:
# import matplotlib.pyplot as plt
import numpy as np 
from pathlib import Path


from optv.parameters import ControlParams, VolumeParams
from optv.calibration import Calibration
from optv.orientation import external_calibration, full_calibration

from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel
from optv.orientation import point_positions
from optv.transforms import convert_arr_pixel_to_metric, distorted_to_flat

# import plotly.express as px
import plotly.figure_factory as ff
# import plotly.graph_objects as go
from scipy.optimize import minimize


In [2]:

# copy of SaveLoad.py
# import numpy as np

def LoadMarkerList(cam):
    return np.loadtxt('markers_c'+str(cam)+'.txt')

def SaveMarkerList(data):
    # Format: x0, y0, x1, y1, x2, y2, x3, y3, X, Y, Z, dX, dY, dZ
    return np.savetxt('markers_error.txt',data)



In [3]:

class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

# load parameter
params = Parameter()



In [4]:
cases_path = (Path.cwd().parent / 'cases')
cases_path.exists()

True

In [5]:
cases = list(cases_path.rglob('case_*'))
for c in cases:
    list_files = list(c.rglob('markers*'))
    list_files.sort()
    print(list_files)


[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c3.txt')]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_allmarkers/markers_c3.txt')]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_extrapolation/markers_c0

In [6]:
cases[0].name.split('case_')[-1]

'interpolation'

In [7]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    cal.set_radial_distortion(x[9:12])
    cal.set_decentering(x[12:14])
    cal.set_affine_trans(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)

    targets = convert_arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
    cpar,
    )
    # err = np.sum(np.abs(xy - targets))
    err = np.sum((xy - targets)**2)
    # print(err)
    return err

def pixel_to_flat(pos, cpar, cal):
    """ converts numpy array of size (2,) from pixel to flat coordinates"""
    pos = convert_arr_metric_to_pixel(np.atleast_2d(pos), cpar)
    distorted_to_flat(pos, cal, pos, 1e-5 )
    return pos

In [8]:
n_cams = len(params.cams)

cpar = ControlParams(n_cams)
cpar.read_control_par(b"parameters/ptv.par")

vpar = VolumeParams()
vpar.read_volume_par(b"parameters/criteria.par")

# Calibration initial guess 

cals = []
for i_cam in range(n_cams):
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    cal.from_file(tmp + b".ori", tmp + b".addpar")
    print(cal.get_pos(), cal.get_angles())
    cals.append(cal)



for case in cases:
    list_files = list(case.rglob('markers*'))
    list_files.sort()
    print(list_files)
    
    # load marker
    markers = [np.loadtxt(_) for _ in list_files]
    
    for c in params.cams:
        print(f" Camera {c}\n")

        XYZ = markers[c][:,2:]
        xy = markers[c][:,:2]
        ID = np.argwhere((XYZ[:,0]>-1))[:,0]

        cal = cals[c]
        # print what you get to see it's still a valid guess
        cal.get_pos(), cal.get_angles()

        
        # We could use this step only if we do not have a good
        # initial guess, but we have one from the previous step

        
        four_points = xy[[0,int(ID.max()/4),int(ID.max()*3/4),ID.max()],:] # choose manually
        ref_pts = XYZ[[0,int(ID.max()/4),int(ID.max()*3/4),ID.max()],:]


        targets = convert_arr_metric_to_pixel(
            image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
        cpar,
        )
        print(f"Before: {four_points - targets}")


        # external_calibration(cal, ref_pts, four_points, cpar)


        x0 = calibration_to_array(cal)
        print(x0)
        sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-11)

        array_to_calibration(sol.x, cal)

        targets = convert_arr_metric_to_pixel(
            image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
        cpar,
        )
        print(f"After: {four_points - targets}")

        targets = convert_arr_metric_to_pixel(
            image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
        cpar,
        )

        # px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
        # fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
        # fig.show()

        # Not sure I understand it correctly, we calibrate with 
        # some markers but always compare with the full set

        case_name = case.name.split('case_')[-1] # 'interpolation', ...

        tmp = [[markers[cam][row,:2] for cam in range(n_cams)] for row in range(len(markers))]
        newtmp = [[pixel_to_flat(pos, cpar, cals[i]) for i, pos in enumerate(row)] for row in tmp]
        newXYZ = []
        for row in newtmp:
            _, pos = point_positions(row, 4, cpar.get_multimedia_params(), cal)
            newXYZ.append(pos)
            
        newXYZ = np.array(newXYZ)

        errors = newXYZ - XYZ
        print(errors)

        # get 3D positions from plate
        # # triangulate 3D position
        # camPs = [ np.asarray([markers[0][i,:2],markers[1][i,:2],markers[2][i,:2],markers[3][i,:2]]) for i in range(len(markers[0]))]
        # P = np.asarray([NewtonSoloff_Triangulation(setP, ax, ay, params)[0] for setP in camPs])

        # # compare error
        # error = XYZ-P
        # output = np.zeros([361*5,8+3+3])
        # for i in range(len(params.cams)):
        #     output[:,2*i:2*(i+1)] = np.loadtxt('markers_c'+str(params.cams[i])+'.txt')[:,:2]
        # output[:,8:11] = XYZ
        # output[:,11:] = error
        # # SaveMarkerList(output)

        # err = np.linalg.norm(error,axis=1)
        # mean_err = np.mean(err)
        # std_err = np.std(err)
        # print('MAE: ', mean_err, ' +- ' , std_err )

        # axis = plt.figure().add_subplot(111, projection='3d')
        # axis.scatter(P[:,2], P[:,0], P[:,1],c='red',s=5)
        # axis.scatter(XYZ[ID,2], XYZ[ID,0], XYZ[ID,1],c='black',s=5)
        # axis.set_xlabel('Z',fontsize=13), axis.set_ylabel('X',fontsize=13), axis.set_zlabel('Y',fontsize=13)
        # plt.tight_layout()
        # plt.show()

        # axis = plt.figure().add_subplot(111, projection='3d')
        # axis.quiver(P[:,2],P[:,0],P[:,1], P[:,2]-XYZ[ID,2], P[:,0]-XYZ[ID,0], P[:,1]-XYZ[ID,1],color='red',length=10,normalize=True)
        # axis.set_xlabel('Z',fontsize=13), axis.set_ylabel('X',fontsize=13), axis.set_zlabel('Y',fontsize=13)
        # plt.tight_layout()
        # plt.show()

[ 322.38251339    4.4040676  1031.76370716] [ 0.1884708   0.20904414 -0.03438548]
[-1.72196260e-01  2.96220478e+02  1.03049543e+03] [-0.2028321  -0.18304368 -0.02799897]
[ 323.84409902  298.102586   1028.23341819] [-0.19496146  0.21075588  0.0246242 ]
[-1.32199240e-01  1.49476157e+01  1.02278223e+03] [ 0.17005024 -0.1784537   0.00354099]
[PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c0.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c1.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c2.txt'), PosixPath('/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/cases/case_interpolation/markers_c3.txt')]
 Camera 0

Before: [[-3.19232797 -0.09974858]
 [ 0.0096724   3.82059895]
 [ 0.56604697 -3.24775243]
 [ 5.05276383  0.65211193]]
[ 3.22382513e+02  4.40406760e+00  1.03176371e+03  1.88470800e-01
  2.09044140e-

TypeError: Argument 'targets' has incorrect type (expected numpy.ndarray, got list)

In [9]:
point_positions?

Docstring:
Calculate the 3D positions of the points given by their 2D projections
using one of the options: 
- for a single camera, uses single_cam_point_positions()
- for multiple cameras, uses multi_cam_point_positions()

Arguments:
np.ndarray[ndim=3, dtype=pos_t] targets - (num_targets, num_cams, 2) array,
    containing the metric coordinates of each target on the image plane of
    each camera. Cameras must be in the same order for all targets.
ControlParams cparam - needed for the parameters of the tank through which
    we see the targets.
cals - a sequence of Calibration objects for each of the cameras, in the
    camera order of ``targets``.
VolumeParams vparam - an object holding observed volume size parameters, needed
    for the single camera case only.

Returns:
res - (n,3) array for n points represented by their targets.
rcm - n-length array, the Ray Convergence Measure for eachpoint for multi camera
    option, or zeros for a single camera option
Type:      builtin_funct